In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
os.getcwd()

'c:\\Users\\Archana\\Desktop\\ML_Project\\ML_COMPLETE_PROJ'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [5]:
from ml_complete_proj.utils.common import read_yaml, create_directories
from ml_complete_proj.utils.common import save_json
from ml_complete_proj.constants import *

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config1 = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        config2 = self.config.mlflow_info

        os.environ['MLFLOW_TRACKING_URI'] = config2.tracking_uri
        os.environ['MLFLOW_TRACKING_USERNAME'] = config2.tracking_username
        os.environ['MLFLOW_TRACKING_PASSWORD'] = config2.tracking_password
        create_directories([config1.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config1.root_dir,
            test_data_path=config1.test_data_path,
            model_path = config1.model_path,
            all_params=params,
            metric_file_name = config1.metric_file_name,
            target_column = schema.name,
            mlflow_uri= config2.tracking_uri,
           
        )

        return model_evaluation_config

In [7]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib


In [8]:
import json
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn

In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mlflow.set_experiment("First Experiment")
        
        mlflow.set_tracking_uri(uri=self.config.mlflow_uri)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

    

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
    
except Exception as e:
    raise e

[2025-01-24 15:32:35,625: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-24 15:32:35,627: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-24 15:32:35,630: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-24 15:32:35,631: INFO: common: created directory at: artifacts]
[2025-01-24 15:32:35,633: INFO: common: created directory at: artifacts/model_evaluation]


2025/01/24 15:32:36 INFO mlflow.tracking.fluent: Experiment with name 'First Experiment' does not exist. Creating a new experiment.


[2025-01-24 15:32:38,759: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\Archana\anaconda3\envs\mlproj\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'ElasticnetModel'.
2025/01/24 15:32:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
